In [73]:
%load_ext autoreload
%autoreload 2
from IPython.display import display
from pkdb_analysis import PKFilter, PKData
from pkdb_analysis.data import PKDataFrame
import copy
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
pkfilter = PKFilter()
for df_key in PKData.KEYS:
    setattr(pkfilter,df_key,{"study_name":"Test1"})
data = PKData.from_db(pkfilter)

In [75]:
test_data = copy.deepcopy(data)


In [71]:
test_data.interventions.df

,study_sid,study_name,intervention_pk,raw_pk,normed,name,route,form,application,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
0,PKDB99999,Test1,2336,2335,True,iv,iv,solution,single dose,0.0,...,midazolam,0.000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram / kilogram
1,PKDB99999,Test1,2335,-1,False,iv,iv,solution,single dose,0.0,...,midazolam,0.075000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg/kg
2,PKDB99999,Test1,2334,2332,True,po15,oral,tablet,single dose,0.0,...,midazolam,0.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
3,PKDB99999,Test1,2333,2331,True,po75,oral,tablet,single dose,0.0,...,midazolam,0.007500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
4,PKDB99999,Test1,2332,-1,False,po15,oral,tablet,single dose,0.0,...,midazolam,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg
5,PKDB99999,Test1,2331,-1,False,po75,oral,tablet,single dose,0.0,...,midazolam,7.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mg


In [72]:
test_data.to_hdf5("test")

/home/janekg/Env/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['study_sid', 'study_name', 'name', 'route', 'form', 'application', 'time_unit', 'measurement_type', 'choice', 'substance', 'unit']]

  """Entry point for launching an IPython kernel.
/home/janekg/Env/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['study_sid', 'study_name', 'individual_name', 'measurement_type', 'choice', 'substance', 'unit']]

  """Entry point for launching an IPython kernel.
/home/janekg/Env/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables wil

# Example queries

## 1. Query study

In [84]:
test_data = copy.deepcopy(data)



In [86]:
test_data.intervention_pk_filter(f_idx_PKDB99999)

In [66]:
def f_idx_PKDB99999(d):
    return (d.study_sid == "PKDB99999")

print(test_data)


------------------------------
PKData (139657458580056)
------------------------------
groups              1  (    6)
individuals         6  (   42)
interventions       6  (    6)
outputs           194  (  194)
timecourses         4  (    4)
------------------------------


In [ ]:
%%timeit -r 2 -n 1
PKDB99999_data = test_data.filter_by_f_idx("interventions", f_idx_PKDB99999)

## Query groups and individuals
### 2.1 Get data for groups with characteristica/keywords X
healthy=True, smoking=N, disease=None,
individual queries and combinations.


In [ ]:
def is_healthy_and_smoker(d): 
    return (d["measurement_type"]=="healthy") & (d["choice"]=="Y")
         

def not_disease(d):
    return  (d["measurement_type"]=="disease")


def smoker_n(d):
    return (d["measurement_type"]=="smoking") & (d["choice"]=="N") \
         & (d["measurement_type"]=="disease")



    
    # health related
    healthy = not d[(d["measurement_type"]=="healthy") & (d["choice"]=="Y")].empty
    not_disease = _is_not(d,"measurement_type", "disease")
    
    # smoking related
    smoker_n = not d[(d["measurement_type"]=="smoking") & (d["choice"]=="N")].empty
    not_smoker_y =  d[(d["measurement_type"]=="smoking") & (d["choice"]=="Y")].empty
    
    return _all(d, *[not_disease,healthy, smoker_n, not_smoker_y])
    
    
    

In [ ]:
test_data = copy.deepcopy(data)


In [ ]:
test_data.groups.groupby("group_pk").get_group(310)


In [ ]:
test_data = copy.deepcopy(data)
test_data.groups = test_data.groups.groupby("group_pk").apply(is_healthy_and_smoker)
test_data.individuals = test_data.individuals.groupby("individual_pk").apply(is_healthy_and_smoker)
test_data._concise()

In [ ]:
test_data.outputs

## 3 Query interventions
### 3.1 Get outputs/timecourses for intervention with substance
intervention with measurement_type "dosing" and substance "caffeine"


In [ ]:
def dosing_and_midazolam(d):
    dosing_and_midazolam = not d[(d["measurement_type"]=="dosing") & (d["substance"]=="midazolam")].empty
    #dosing = not d[(d["measurement_type"]=="dosing")].empty
    #caffeine = not d[(d["substance"]=="midazolam")].empty
    
    return _all(d, *[dosing_and_midazolam])


### 3.2 Get outputs/timecourses where multiple interventions were given

In [ ]:
test_data = copy.deepcopy(data)
test_data.interventions = test_data.interventions.groupby("pk").apply(dosing_and_midazolam)
test_data._concise()

In [ ]:
test_data._len

In [ ]:
test_data.interventions.head()


## 4 Query outputs/timecourses
### 4.1 query by measurement_type
query all auc_inf

In [ ]:

def is_auc_inf(d):
    auc_inf = _is(d,"measurement_type","auc_inf" )
    return _all(d, *[auc_inf])
   
    

## 5 Other Query others
### 5.1 Complex
get clearance of codeine for all subjects, that have been phenotyped. 


In [ ]:
def is_phenotyping(d):
    metabolic_phenotype = _is(d,"measurement_type","metabolic phenotype")
    metabolic_ratio = _is(d,"measurement_type","metabolic ratio")
    return _any(d, *[metabolic_phenotype,metabolic_ratio])

In [ ]:
test_data.outputs.head()

In [ ]:
test_data = copy.deepcopy(data)
outputs = test_data.outputs.groupby("output_pk").apply(is_phenotyping)

In [ ]:
# all substances related to metabolic phenotype and metabolic ratio. I am only interessted in cyp2d6 phenotyping -> substance = 'spar/(2hspar+5hspar)', 'deb/4hdeb', 'dtf/dmt'
outputs.substance.unique()

In [ ]:
def is_cyp2d6_phenotyping(d):
    cyp2d6_phenotype_substances = ['spar/(2hspar+5hspar)', 'deb/4hdeb', 'dtf/dmt']
    cyp2d6_phenotype_measurement_types = ["metabolic phenotype","metabolic ratio"]
    cyp2d6_phenotype = not d[(d["substance"].isin(cyp2d6_phenotype_substances)) & (d["measurement_type"].isin(cyp2d6_phenotype_measurement_types))].empty
    return _any(d, *[cyp2d6_phenotype])
                                                               
def codeine_clearance(d):
    codeine_clearance = not d[(d["measurement_type"]=="clearance") & (d["substance"]=="codeine")].empty                                                           
    return _all(d, *[codeine_clearance])                                                               

In [ ]:
cyp2d6_phenotype_data = copy.deepcopy(data)                                                   
cyp2d6_phenotype_outputs = test_data.outputs.groupby("output_pk").apply(is_cyp2d6_phenotyping)

In [ ]:
cyp2d6_phenotype_groups = cyp2d6_phenotype_outputs["group_pk"].unique()
cyp2d6_phenotype_individuals = cyp2d6_phenotype_outputs["individual_pk"].unique()

cyp2d6_phenotype_data.groups = cyp2d6_phenotype_data.groups[cyp2d6_phenotype_data.groups["group_pk"].isin(cyp2d6_phenotype_groups)]
cyp2d6_phenotype_data.individuals = cyp2d6_phenotype_data.individuals[cyp2d6_phenotype_data.individuals["individual_pk"].isin(cyp2d6_phenotype_individuals)]
cyp2d6_phenotype_data._concise()

cyp2d6_phenotype_data.outputs = cyp2d6_phenotype_data.outputs.groupby("output_pk").apply(codeine_clearance)

In [ ]:
cyp2d6_phenotype_data._concise()


In [ ]:
cyp2d6_phenotype_data._len

In [ ]:
cyp2d6_phenotype_data.outputs